In [1]:
from utils_1551 import Corpus, load_train_and_test

from bpemb import BPEmb
from uk_stopwords import STOP_WORDS
from typing import List, Dict
import numpy as np

In [2]:
train, test = load_train_and_test()

In [3]:
dim = 300
bpemb_ua = BPEmb(lang="uk", dim=dim)

In [4]:
print(bpemb_ua.embed('щось там').shape)
print(bpemb_ua.encode('щось там'))

(3, 300)
['▁що', 'сь', '▁там']


In [5]:
import re
re_tokens = re.compile("([\w][\w]*'?\w?)")

def text2tokens(text: str)->List[str]:
    return re_tokens.findall(text)

s = "Ваша відповідь:  Повідомляємо, що відповідно до п. 5.7 Державного стандарту України 4123-2006 «Пристрій примусового зниження швидкості дорожньо-транспортної техніки на вулицях і дорогах. Загальні технічні вимоги» заборонено встановлювати пристрої примусового зниження швидкості руху транспортних засобів на магістральних вулицях, а просп. Григоренка має категорію магістральної вулиці районного значення."
print(text2tokens(s))

['Ваша', 'відповідь', 'Повідомляємо', 'що', 'відповідно', 'до', 'п', '5', '7', 'Державного', 'стандарту', 'України', '4123', '2006', 'Пристрій', 'примусового', 'зниження', 'швидкості', 'дорожньо', 'транспортної', 'техніки', 'на', 'вулицях', 'і', 'дорогах', 'Загальні', 'технічні', 'вимоги', 'заборонено', 'встановлювати', 'пристрої', 'примусового', 'зниження', 'швидкості', 'руху', 'транспортних', 'засобів', 'на', 'магістральних', 'вулицях', 'а', 'просп', 'Григоренка', 'має', 'категорію', 'магістральної', 'вулиці', 'районного', 'значення']


In [6]:
def text2vec(text: str):
    res = np.zeros(dim)
    words = text2tokens(text.lower())
    for w in words:
        if w in STOP_WORDS:
            continue
        for v in bpemb_ua.embed(w):
            res += v
    res /= max(np.linalg.norm(res), 1e-6)
    return res

print(text2vec(s).shape)

(300,)


In [13]:
def corpus2xy(corpus: Corpus)->(np.array, np.array):
    x, y = [], []
    for name, messages in corpus.items():
        for id_, text in messages:
            vec = text2vec(text)
            x.append(vec)
            y.append(name)
    return np.array(x), np.array(y)

def extract_labels(corpus: Corpus)->Dict[str, int]:
    res = {name:i for i, name in enumerate(corpus.keys())}
    return res

labels_dict = extract_labels(train)
def labels_to_ind(y: np.array)->np.array:
    return np.array([labels_dict[name] for name in y])

In [16]:
x_train, y_train = corpus2xy(train)

In [15]:
y_train_idx = labels_to_ind(y_train)

In [17]:
print(x_train.shape, x_train.dtype)
print(y_train_idx.shape, y_train_idx.dtype)

(48120, 300) float64
(48120,) int32


In [18]:
x_test, y_test = corpus2xy(test)
y_test_idx = labels_to_ind(y_test)

In [25]:
from lightgbm import LGBMClassifier
clf = LGBMClassifier(objective='multiclass', random_state=1974, min_child_samples=5)
clf.fit(x_train, y_train_idx)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=5, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective='multiclass',
        random_state=1974, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [26]:
y_pred = clf.predict(x_test)

In [27]:
from sklearn.metrics import classification_report

print(classification_report(y_test_idx, y_pred))

C:\Users\ssotn\Anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        78
           1       0.00      0.00      0.00       182
           2       0.00      0.00      0.00        73
           3       0.00      0.00      0.00        19
           4       0.00      0.00      0.00        22
           5       0.00      0.00      0.00        25
           6       0.00      0.00      0.00        15
           7       0.00      0.00      0.00       161
           8       0.00      0.00      0.00        33
           9       0.00      0.00      0.00        24
          10       0.00      0.00      0.00        22
          11       0.00      0.00      0.00        82
          12       0.00      0.00      0.00        23
          13       0.00      0.00      0.00        34
          14       0.00      0.00      0.00        55
          15       0.00      0.00      0.00        16
          16       0.00      0.00      0.00       109
          17       0.00    